<a href="https://colab.research.google.com/github/ejmata2/python/blob/main/PyCuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial de PyCuda

## Instalación 


In [2]:
!pip install pycuda

     |████████████████████████████████| 1.7MB 8.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627104 sha256=2726d777bdae4048c369848a3e134f954026b6c2ba2319ba7e36bca560f2354f
  Stored in directory: /root/.cache/pip/wheels/d5/55/64/fd4dddcc5f1c25eebd90b5291c3769101dc978c70165685512
Successfully built pycuda
  Created wheel for pytools: filename=pytools-2021.2.6-py2.py3-none-any.whl size=60643 sha256=eead12be33bf022270d3e060bbc7bb5df786c88d9b5cb1b165f5b462dcca5f23
  Stored in directory: /root/.cache/pip/wheels/8c/a6/65/447b9b4fd1d9bde84ad2fea2431a38f69f3fb573476a98ae03
Successfully built pytools


Before you can use PyCuda, you have to import and initialize it:

In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In PyCuda, you will mostly transfer data from numpy arrays on the host. Let’s make a 4x4 array of random numbers:

In [4]:
import numpy
a = numpy.random.randn(4,4)

a consists of double precision numbers, but most nVidia devices only support single precision:

In [5]:
a = a.astype(numpy.float32)

 allocate memory on the device

In [6]:
a_gpu = cuda.mem_alloc(a.nbytes)

 transfer the data to the GPU:

In [7]:
cuda.memcpy_htod(a_gpu, a)

# Executing a Kernel

For this tutorial, we’ll stick to something simple: We will write code to double each entry in a_gpu. To this end, we write the corresponding CUDA C code, and feed it into the constructor of a pycuda.compiler.SourceModule:

In [8]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

If there aren’t any errors, the code is now compiled and loaded onto the device. We find a reference to our pycuda.driver.Function and call it, specifying a_gpu as the argument, and a block size of 4x4:

In [9]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

Finally, we fetch the data back from the GPU and display it, together with the original a:

In [11]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[ 2.985099   -2.987172    4.665507   -2.0345824 ]
 [-0.5008996  -5.4453635   2.6517894  -1.3422637 ]
 [ 0.13100177  1.7423439  -2.8372016  -3.248441  ]
 [-2.0961182   2.1598656  -0.7870664   2.3708656 ]]
[[ 1.4925495  -1.493586    2.3327534  -1.0172912 ]
 [-0.2504498  -2.7226818   1.3258947  -0.67113185]
 [ 0.06550089  0.87117195 -1.4186008  -1.6242205 ]
 [-1.0480591   1.0799328  -0.3935332   1.1854328 ]]
